# Plotly Image Slicer Example

Conda install requires: `floatview, ipywidgets, matplotlib, numpy, plotly, astropy, jupyter`

The `floatview` might have to be pip installed and there are further installation instructions here https://github.com/denphi/jupyterlab-floatview.

```
pip install floatview
jupyter labextension install @jupyterlab/plotly-extension@0.18.1
jupyter labextension install plotlywidget@0.6.0
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install jupyterlab-floatview
```



In [24]:
from floatview import Floatview
from ipywidgets import IntSlider, Dropdown
import ipywidgets
import matplotlib.pyplot as p
import numpy as np
import plotly.graph_objs as go

from astropy.io import fits

class Viewer:
    
    def __init__(self):
        
        # Load the data
        f = fits.open('/home/craig/Documents/STScI/cubeviz/data/manga-7495-12704-LOGCUBE.fits.gz')
        self._fits_data = f
        
        self._current_slice = None
        self._extensions = ('FLUX', 'IVAR', 'MASK')
        self._current_extension = self._extensions[0]
                
        self.create_image()
        
        # Create the float view
        self._sc = Floatview(title='Image View', mode='split-right')
        
        # Create Extension drop down
        self._dropdown_extension = Dropdown(options=(self._extensions), value=self._extensions[0], description='Extension:', disabled=False)
        self._dropdown_extension.observe(self._dropdown_extension_on_value_change)

        # Create slice slider
        self._slider_slice = IntSlider(description='Slice #:', max=self._fits_data[self._current_extension].data.shape[0])
        self._slider_slice.observe(self._slider_slice_on_value_change)
        
        # Add figure to widget
        self._fig = go.FigureWidget(self._gofig)
        
        # Display the float view
        with self._sc:
            display(ipywidgets.widgets.VBox([self._fig, self._dropdown_extension, self._slider_slice]))
                
    def create_image(self):
        
        self._current_slice = 0
        
        # Add the image data
        self._trace1 = {
          "x": np.arange(74),
          "y": np.arange(74), 
          "z": self._fits_data[self._current_extension].data[0], 
          "showlegend": False, 
          "type": "heatmap",
          "hoverinfo": "all"
        }
        self._data = go.Data([self._trace1])
        
        # Plot Layout information
        layout = {
          "legend": {
            "x": 1, 
            "y": 0.5, 
            "bgcolor": "rgb(255,255,255)"
          }, 
          "margin": {"r": 10}, 
          "paper_bgcolor": "rgb(255,255,255)", 
          "plot_bgcolor": "rgb(229,229,229)", 
          "showlegend": True, 
          "xaxis": {
            "gridcolor": "rgb(255,255,255)", 
            "showgrid": True, 
            "showline": False, 
            "showticklabels": True, 
            "tickcolor": "rgb(127,127,127)", 
            "ticks": "outside", 
            "title": "x", 
            "type": "linear", 
            "zeroline": False
          }, 
          "yaxis": {
            "gridcolor": "rgb(255,255,255)", 
            "showgrid": True, 
            "showline": False, 
            "showticklabels": True, 
            "tickcolor": "rgb(127,127,127)", 
            "ticks": "outside", 
            "title": "y", 
            "type": "linear", 
            "zeroline": False,
            "scaleanchor": "x"
          }
        }
        
        self._gofig = go.Figure(data=self._data, layout=layout)

    def _slider_slice_on_value_change(self, change):
        """
        Slice slider call back
        """
       
        # Get the new value
        sl = change['new']['value']
        
        # If current, stop, else update
        if sl == self._current_slice:
            return
        else:
            self._current_slice = slice
        
        # Get the data and update the figure
        td = self._fits_data[self._current_extension].data[sl]
        self._fig.data[0].update({'z': td})
        
    def _dropdown_extension_on_value_change(self, change):
        """
        FITS Extension dropdown call back
        """
       
        if change['type'] == 'change':
            self._current_extension = self._extensions[change['new']['index']]

        # Get the data and update the figure
        td = self._fits_data[self._current_extension].data[sl]
        self._fig.data[0].update({'z': td})
    
v = Viewer()

## Create a Second Viewer

In [20]:
v2 = Viewer()

Now try moving the viewers around to create a tiled look.